In [ ]:
import wandb
from openai import OpenAI

In [ ]:
import os
# os.environ['http_proxy']="http://proxy61.iitd.ac.in:3128"
# os.environ['https_proxy']="http://proxy61.iitd.ac.in:3128"

In [ ]:
from config.read_config_other import read_config
import torch

# Reading config file
config = read_config("config/config_other.txt")

In [ ]:
# setting env variables
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
os.environ['WANDB_NOTEBOOK_NAME'] = 'final_'+ config['llm_name']

In [ ]:
# get the run number
fp = open("run_seq.txt",'r')
read_run_data = fp.readline().strip()

run_data = {}

while(read_run_data):
    key,value = read_run_data.split(" ")
    run_data[key]=int(value)
    read_run_data= fp.readline().strip()

fp.close()

In [ ]:
# new run number
if config['llm_name'] not in run_data.keys():
    run_data[config['llm_name']] = 0

run_number = str(run_data[config['llm_name']]+1)
run_data[config['llm_name']] += 1

fp = open("run_seq.txt",'w')

for i in run_data.keys():
    fp.write(i+" "+str(run_data[i])+"\n")
fp.close()

In [ ]:
model_name = "final_"+config['llm_name']+"_"+config['sql_dataset'].strip().split(".")[0].strip()+"_method_2"
model_tag = config['llm_name']

run = wandb.init(
    project=model_name, 
    notes="Run number "+run_number,
    tags=["openai",model_tag]
)

wandb.config = {"Parameters":model_tag}

In [ ]:
prediction_table = wandb.Table(columns=["Prompt","Response","F_prompt","F_response"])

In [ ]:
def read_dataset_file(file_loc):
    data_in_file = []

    file_l = file_loc
    fp = open(file_l,'r')
    sentence = fp.readline()
    while sentence:
        data_in_file.append(sentence)
        sentence = fp.readline()
    
    return data_in_file

dataset_file = config['dataset_path'] + config['sql_dataset']
dataset_file_id = config['dataset_path']+config['db_id']

dataset_file_read = read_dataset_file(dataset_file)
dataset_file_id_read = read_dataset_file(dataset_file_id)
samples = read_dataset_file(config['samples_path']+config['sampling_file'])

samples_list = samples[0].strip().split(' ')

for i in range(len(samples_list)):
    samples_list[i] = int(samples_list[i])

In [ ]:
# Reading table create commands

def read_file(file_loc):
    sentence = ""
    file_l = file_loc
    fp = open(file_l,'r')
    sentence = fp.read()
    fp.close()
    return sentence

In [ ]:
# Reading the prompt files
prompt = {}

for i in range(1,int(config['prompt_count'])+1):
    prompt_file_loc = config['prompt_path'] + config['prompt_file_'+str(i)]
    prompt_data = read_file(prompt_file_loc)

    prompt['prompt_file_'+str(i)] = prompt_data

print(prompt)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_path = config['llm_path']+config['llm_name']

tokenizer = AutoTokenizer.from_pretrained(model_path)

model = AutoModelForCausalLM.from_pretrained(
        model_path,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        device_map="auto",
        use_cache=True,
        # load_in_8bit=True,
        # load_in_4bit=True,
    )

In [ ]:
# make sure the model stops generating at triple ticks
# eos_token_id = tokenizer.convert_tokens_to_ids(["```"])[0]
eos_token_id = tokenizer.eos_token_id
max_new_tokens = 1000
num_beam = 1
do_sample = False
return_full_text = False

pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        return_full_text=return_full_text, # added return_full_text parameter to prevent splitting issues with prompt
        num_beams=num_beam, # do beam search with 5 beams for high quality results
    )

In [ ]:
import pandas as pd

def creatiing_schema(DATASET_JSON):
    schema_df = pd.read_json(DATASET_JSON)
    schema_df = schema_df.drop(['column_names','table_names'], axis=1)
    schema = []
    f_keys = []
    p_keys = []
    for index, row in schema_df.iterrows():
        tables = row['table_names_original']
        col_names = row['column_names_original']
        col_types = row['column_types']
        foreign_keys = row['foreign_keys']
        primary_keys = row['primary_keys']
        for col, col_type in zip(col_names, col_types):
            index, col_name = col
            if index == -1:
                for table in tables:
                    schema.append([row['db_id'], table, '*', 'text'])
            else:
                schema.append([row['db_id'], tables[index], col_name, col_type])
        for primary_key in primary_keys:
            index, column = col_names[primary_key]
            p_keys.append([row['db_id'], tables[index], column])
        for foreign_key in foreign_keys:
            first, second = foreign_key
            first_index, first_column = col_names[first]
            second_index, second_column = col_names[second]
            f_keys.append([row['db_id'], tables[first_index], tables[second_index], first_column, second_column])
    spider_schema = pd.DataFrame(schema, columns=['Database name', ' Table Name', ' Field Name', ' Type'])
    spider_primary = pd.DataFrame(p_keys, columns=['Database name', 'Table Name', 'Primary Key'])
    spider_foreign = pd.DataFrame(f_keys,
                        columns=['Database name', 'First Table Name', 'Second Table Name', 'First Table Foreign Key',
                                 'Second Table Foreign Key'])
    return spider_schema,spider_primary,spider_foreign

In [ ]:
DATASET_SCHEMA = config['dataset_path']+config['dataset_schema']
spider_schema,spider_primary,spider_foreign = creatiing_schema(DATASET_SCHEMA)

In [ ]:
def find_foreign_keys_MYSQL_like(db_name):
  df = spider_foreign[spider_foreign['Database name'] == db_name]
  output = "[ "
  for index, row in df.iterrows():
    output += row['First Table Name'] + '.' + row['First Table Foreign Key'] + " = " + row['Second Table Name'] + '.' + row['Second Table Foreign Key'] + ', '
  output= output[:-2] + " ]"
  return output
def find_fields_MYSQL_like(db_name):
  df = spider_schema[spider_schema['Database name'] == db_name]
  df = df.groupby(' Table Name')
  output = ""
  for name, group in df:
    output += "Table " +name+ ', columns = [ '
    for index, row in group.iterrows():
      output += row[" Field Name"]+', '
    output = output[:-2]
    output += " ]\n"
  return output
def find_primary_keys_MYSQL_like(db_name):
  df = spider_primary[spider_primary['Database name'] == db_name]
  output = "[ "
  for index, row in df.iterrows():
    output += row['Table Name'] + '.' + row['Primary Key'] +', '
  output = output[:-2]
  output += " ]\n"
  return output

In [ ]:
from create_prompt import create_prompt
import time 

prediction_table_temp = []

if not os.path.exists("input_1"):
    os.mkdir("input_1")

if not os.path.exists("input_1/"+model_name):
    os.mkdir("input_1/"+model_name)

if not os.path.exists("input_1/"+model_name+"/"+run_number):
    os.mkdir("input_1/"+model_name+"/"+run_number)

if not os.path.exists("output_1"):
    os.mkdir("output_1")

if not os.path.exists("output_1/"+model_name):
    os.mkdir("output_1/"+model_name)

if not os.path.exists("output_1/"+model_name+"/"+run_number):
    os.mkdir("output_1/"+model_name+"/"+run_number)

start_time = time.time()

for idx, i in enumerate(samples_list):

    database = dataset_file_id_read[i].strip()
    # print(database)

    fields = find_fields_MYSQL_like(database)+"\n"
    fields += "Foreign_keys = " + find_foreign_keys_MYSQL_like(database) + '\n'
    fields += "Primary_keys = " + find_primary_keys_MYSQL_like(database)

    q1 = "[SQL_1] "+dataset_file_read[(i)*2].strip()
    if q1[-1]!=";":
        q1+=";"
    
    q2 = "[SQL_2] "+dataset_file_read[(i)*2+1].strip()
    if q2[-1]!=";":
        q2+=";"

    # creating prompt
    # message = create_prompt([tbl_list,'\n',q1,q2,'\n',prompt['prompt_file_1']],create_table,0)
    message = create_prompt([prompt['prompt_file_1'],'\n',fields,'\n',prompt['prompt_file_2'],'\n',q1,'\n',q2,'\n\n',prompt['prompt_file_3'],'\n\n',prompt['prompt_file_4']],[],-1)
    print(message)

    # message.append({"role":"user", "content":prompt_temp_1})

    response = ( pipe
                ( message, 
                 num_return_sequences=1, 
                 eos_token_id=eos_token_id, 
                 pad_token_id=eos_token_id,
                 )[0]["generated_text"])
                # .split(";")[0]).split("```")[0].strip() + ";" )
    
    response = response.strip().split("### Task")[0].strip()
    response = response.strip().split("### SQL")[0].strip()
    print(response)
    
    fp = open("input_1/"+model_name+"/"+run_number+"/query_"+str(i+1)+".txt",'w')
    fp.write(str(message))
    fp.close()

    fp = open("output_1/"+model_name+"/"+run_number+"/output_"+str(i+1)+".txt",'w')
    fp.write(str(response))
    fp.close()

    prediction_table_temp.append([str(message),str(response)])


end_time = time.time()
inf_time = end_time - start_time

In [ ]:
openai_organization = ""
openai_api_key = ""

In [ ]:
os.environ['OPENAI_API_KEY'] = openai_api_key
client = OpenAI()

In [ ]:
import time 

if not os.path.exists("final_result_input"):
    os.mkdir("final_result_input")

if not os.path.exists("final_result_input/"+model_name):
    os.mkdir("final_result_input/"+model_name)

if not os.path.exists("final_result_input/"+model_name+"/"+run_number):
    os.mkdir("final_result_input/"+model_name+"/"+run_number)

if not os.path.exists("final_result_output"):
    os.mkdir("final_result_output")

if not os.path.exists("final_result_output/"+model_name):
    os.mkdir("final_result_output/"+model_name)

if not os.path.exists("final_result_output/"+model_name+"/"+run_number):
    os.mkdir("final_result_output/"+model_name+"/"+run_number)

start_time = time.time()

for idx, i in enumerate(samples_list):
    prompt_temp_2 = create_prompt([prompt['prompt_file_5'],'\n',read_file("output_1/"+model_name+"/"+run_number+"/output_"+str(i+1)+".txt"),'\n',prompt['prompt_file_4']], [], -1)
    print(prompt_temp_2)

    message = [{'role':'user', 'content':prompt_temp_2}]

    output =  response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages = message,
        temperature=0.02,
        max_tokens=1000,
        frequency_penalty=0.0
    )

    result = output.choices[0].message.content
    print(result)

    fp = open("final_result_input/"+model_name+"/"+run_number+"/query_"+str(i+1)+".txt",'w')
    fp.write(prompt_temp_2)
    fp.close()

    fp = open("final_result_output/"+model_name+"/"+run_number+"/output_"+str(i+1)+".txt",'w')
    fp.write(result)
    fp.close()

    prediction_table_temp[idx].append(prompt_temp_2)
    prediction_table_temp[idx].append(result)

end_time = time.time()
res_time = end_time - start_time
    

In [ ]:
for i in prediction_table_temp:
    prediction_table.add_data(i[0],i[1],i[2],i[3])

wandb.log({'predictions': prediction_table})
wandb.finish()

In [ ]:
path = "final_result_output/"+model_name+"/"+run_number+"/output_"

In [ ]:
if not os.path.exists("final_output_count"):
     os.mkdir("final_output_count")

if not os.path.exists("final_output_count/"+model_name):
    os.mkdir("final_output_count/"+model_name)
    
if not os.path.exists("final_output_count/"+model_name+"/"+run_number):
    os.mkdir("final_output_count/"+model_name+"/"+run_number)

yes_cnt = 0
no_cnt = 0
no_idea = 0
for idx, i in enumerate(samples_list):
    fp = open(path+str(i+1)+".txt", 'r')
    data = fp.read()

    data = data.lower().replace("  "," ")
    if "non equivalent" in data or "non-equivalent" in data or "not equivalent" in data or "not-equivalent" in data or "nonequivalent" in data or "notequivalent" in data:
        no_cnt+=1
    elif "equivalent" in data:
        yes_cnt+=1
    else:
        print(i+1)
        print(data)
        no_idea +=1

fp = open("final_output_count/"+model_name+"/"+run_number+"/result.txt",'w')
fp.write("yes: "+str(yes_cnt)+"\n")
fp.write("no: "+str(no_cnt)+"\n")
fp.write("no idea: "+str(no_idea)+"\n")
fp.write("Inference time: "+ str(inf_time) +"\n")
fp.write("Result computation time: "+ str(res_time)+ "\n")
fp.write("####### model details ####### \n")
fp.write("model_name: "+str(model_name)+"\n")
fp.write("max_new_tokens: "+str(max_new_tokens)+"\n")
fp.write("num_beam: "+ str(num_beam)+"\n")
fp.write("do_sample"+ str(do_sample) + "\n")
fp.write("return_full_text"+ str(return_full_text))

fp.close()


In [ ]:
print('yes',yes_cnt)
print('no',no_cnt)
print('no idea',no_idea)